In [3]:
import pandas as pd
import numpy as np
import os
import shutil

##### Selección de estaciones.
Filtrar las estaciones que cumplen con:
* Completitud: Que los datos faltantes sean < 10%.
* Calidad Mensual: Que no haya meses con más de 5 días faltantes (estos se vuelven NaN).

In [ ]:
# --- CONFIGURACIÓN ---
PATH_ENTRADA = "../01_DATA/SMN_limpio/91-20"
PATH_SALIDA = "../01_DATA/for_indices"
FECHA_INICIO = "1991-01-01"
FECHA_FIN = "2020-12-31"

def tiene_rachas_nan(serie, max_consecutivos=5):
    # Creamos una serie booleana (True donde es NaN)
    es_nan = serie.isna()
    # Identificamos bloques consecutivos de True
    # (Comparamos el valor con el anterior para marcar cambios de bloque)
    bloques = es_nan.ne(es_nan.shift()).cumsum()
    # Contamos el tamaño de cada bloque de NaNs
    tamaño_bloques = es_nan.groupby(bloques).transform('sum')
    # Si algún bloque de NaNs es >= al límite, devolvemos True
    return (tamaño_bloques >= max_consecutivos).any()

def filtrar_estaciones_estricto():
    archivos = [f for f in os.listdir(PATH_ENTRADA) if f.endswith('.csv')]
    aptas = 0
    
    print(f"Iniciando filtrado estricto de {len(archivos)} estaciones...")

    for archivo in archivos:
        df = pd.read_csv(os.path.join(PATH_ENTRADA, archivo), 
                         index_col='FECHA', parse_dates=True)
        
        # 1. Recorte temporal
        df_p = df.loc[FECHA_INICIO : FECHA_FIN]
        
        # 2. Validación de existencia de datos
        if df_p.empty or len(df_p) < 1000: # Filtro básico de seguridad
            continue

        # --- CRITERIOS DE CALIDAD ---
        
        # A. Máximo 10% de faltantes totales
        pct_nan = (df_p['PRECIP'].isna().sum() / len(df_p)) * 100
        
        # B. No más de 5 días SEGUIDOS con NaN
        racha_nan = tiene_rachas_nan(df_p['PRECIP'], max_consecutivos=15)
        
        # C. No más de 5 días faltantes en UN MES (alternos o no)
        # Esto asegura que cada mes sea representativo para el SPI
        meses_invalidos = (df_p['PRECIP'].isna().resample('MS').sum() > 15).any()

        # --- DECISIÓN FINAL ---
        if pct_nan <= 20 and not racha_nan and not meses_invalidos:
            ruta_destino = os.path.join(PATH_SALIDA, archivo)
            df_p.to_csv(ruta_destino)
            aptas += 1
            print(f"✅ {archivo}: APTO (Faltantes: {pct_nan:.1f}%)")
        else:
            razon = []
            if pct_nan > 20: razon.append(f"Total NaNs > 20% ({pct_nan:.1f}%)")
            if racha_nan: razon.append("Racha >= 15 días NaN")
            if meses_invalidos: razon.append("Meses con 15 5 NaNs")
            # print(f"❌ {archivo}: Descartado por {', '.join(razon)}")

    print(f"\n--- FILTRADO TERMINADO ---")
    print(f"Estaciones finales en 'for_indices': {aptas}")

filtrar_estaciones_estricto()

Iniciando filtrado estricto de 125 estaciones...
✅ SMN_14002_1991_2020.csv: APTO (Faltantes: 0.0%)
✅ SMN_14038_1991_2020.csv: APTO (Faltantes: 0.0%)
✅ SMN_14066_1991_2020.csv: APTO (Faltantes: 0.1%)
✅ SMN_14339_1991_2020.csv: APTO (Faltantes: 0.1%)

--- FILTRADO TERMINADO ---
Estaciones finales en 'for_indices': 4


In [12]:
# --- CONFIGURACIÓN ---
# Asegúrate de que estas rutas sean correctas en tu computadora
PATH_ENTRADA = "../01_DATA/SMN_limpio/91-20"
PATH_SALIDA = "../01_DATA/for_indices"
FECHA_INICIO = "1991-01-01"
FECHA_FIN = "2020-12-31"
MIN_AÑOS_UTILES = 20  # Mínimo de años con datos para que el SPI tenga sentido

def filtrar_estaciones_final():
    # 1. Obtener lista de archivos
    archivos = [f for f in os.listdir(PATH_ENTRADA) if f.endswith('.csv')]
    
    if not archivos:
        print(f"❌ Error: No se encontraron archivos CSV en {PATH_ENTRADA}")
        return

    print(f"Analizando {len(archivos)} archivos...")
    aptas = 0
    
    for archivo in archivos:
        # 2. Cargar datos
        df = pd.read_csv(os.path.join(PATH_ENTRADA, archivo), 
                         index_col='FECHA', parse_dates=True)
        
        # 3. Recorte al periodo de estudio
        df_p = df.loc[FECHA_INICIO : FECHA_FIN].copy()
        
        if df_p.empty:
            continue

        # --- LÓGICA DE CALIDAD ---
        
        # A. Identificar meses malos (> 5 días con NaN)
        # En lugar de tirar la estación, invalidamos el mes
        faltantes_por_mes = df_p['PRECIP'].isna().resample('MS').sum()
        mask_meses_malos = faltantes_por_mes > 5
        
        # B. Calcular cuántos "Años Útiles" quedan realmente
        meses_totales = len(faltantes_por_mes)
        meses_buenos = (~mask_meses_malos).sum()
        años_utiles = meses_buenos / 12
        
        # --- DECISIÓN ---
        # Solo pedimos que la estación tenga suficiente "sustancia" histórica
        if años_utiles >= MIN_AÑOS_UTILES:
            # OPCIONAL: Si quieres "limpiar" los meses malos antes de guardar:
            # for fecha_mes en faltantes_por_mes[mask_meses_malos].index:
            #     df_p.loc[fecha_mes : fecha_mes + pd.offsets.MonthEnd(0), 'PRECIP'] = np.nan
            
            ruta_destino = os.path.join(PATH_SALIDA, archivo)
            df_p.to_csv(ruta_destino)
            aptas += 1
            print(f"✅ {archivo}: APTO. Años útiles: {años_utiles:.1f}")
        else:
            print(f"⚠️ {archivo}: Descartado. Solo tiene {años_utiles:.1f} años útiles.")

    print(f"\n--- PROCESO TERMINADO ---")
    print(f"Estaciones guardadas en '{PATH_SALIDA}': {aptas}")

# EJECUTAR
filtrar_estaciones_final()

Analizando 125 archivos...
✅ SMN_14002_1991_2020.csv: APTO. Años útiles: 30.0
✅ SMN_14006_1991_2020.csv: APTO. Años útiles: 29.8
✅ SMN_14009_1991_2020.csv: APTO. Años útiles: 27.6
✅ SMN_14011_1991_2020.csv: APTO. Años útiles: 28.3
✅ SMN_14016_1991_2020.csv: APTO. Años útiles: 28.7
✅ SMN_14017_1991_2020.csv: APTO. Años útiles: 25.5
✅ SMN_14018_1991_2020.csv: APTO. Años útiles: 28.3
✅ SMN_14023_1991_2020.csv: APTO. Años útiles: 29.8
✅ SMN_14024_1991_2020.csv: APTO. Años útiles: 26.0
✅ SMN_14026_1991_2020.csv: APTO. Años útiles: 27.6
✅ SMN_14028_1991_2020.csv: APTO. Años útiles: 25.2
✅ SMN_14029_1991_2020.csv: APTO. Años útiles: 29.2
✅ SMN_14030_1991_2020.csv: APTO. Años útiles: 28.2
✅ SMN_14032_1991_2020.csv: APTO. Años útiles: 25.7
✅ SMN_14033_1991_2020.csv: APTO. Años útiles: 24.2
✅ SMN_14034_1991_2020.csv: APTO. Años útiles: 29.5
✅ SMN_14035_1991_2020.csv: APTO. Años útiles: 27.4
✅ SMN_14036_1991_2020.csv: APTO. Años útiles: 29.6
⚠️ SMN_14037_1991_2020.csv: Descartado. Solo tiene 16.7